Este es un cuaderno simple para trabajar con algunas imágenes de galaxias y aplicar el algoritmo de agrupamiento KMeans.

Acompaña al Capítulo 7 del libro (3 de 4) pero solo aparece en la sec. 7.6.

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman. Traducido por Manuel Pichardo Marcano y Genaro Suárez.

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline

import skimage
from skimage.transform import resize, rescale
from skimage import io
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

Este conjunto de datos está compuesto por 200 imágenes seleccionadas al azar del desafío Kaggle Galaxy Zoo:

https://www.kaggle.com/c/galaxy-zoo-the-galaxy-challenge

El siguiente código visualiza los primeros 25 objetos en nuestro conjunto de datos. Podemos ejecutarlo para obtener una vista de las primeras 25 galaxias. Nota: es posible que reciba un mensaje de error, en este caso, consulte aquí


https://stackoverflow.com/questions/43288550/iopub-data-rate-exceeded-in-jupyter-notebook-when-viewing-image

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(ax.shape[0]):

    img = skimage.io.imread('../data/galaxy_images/Image_'+str(i)+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

    

Ahora deshagámonos de algunas fuentes múltiples.

In [ ]:
#Esto muestra cómo se pueden identificar y enmascarar múltiples fuentes.

n_ob = 5

fig, ax = plt.subplots(2, n_ob, figsize=(50, 20))

for i in range(n_ob):

    img = skimage.io.imread('../data/galaxy_images/Image_'+str(i)+'.png')

    image_gray = rgb2gray(img)

    blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

    # Compute radii in the 3rd column.
    
    blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
    
    blobs_log = blobs_log[blobs_log[:,2].argsort()[::-1]]
    
    ax[0,i].imshow(img, interpolation='nearest')

    X, Y = np.ogrid[:img.shape[0], :img.shape[1]]
    
    center = np.array([img.shape[0]/2, img.shape[1]/2]) #center
    
    for blob in blobs_log:    
        y, x, r = blob    
        c = plt.Circle((x, y), r, color = 'yellow', linewidth=2, fill=False)
        ax[0,i].add_patch(c)
        
        if (np.linalg.norm(np.array([x,y])-center)) > 10: #If not in center
        
            mask = (X - blob[0])**2 + (Y - blob[1])**2 < r**2
            img[mask] = 0
    
    ax[1,i].imshow(img, interpolation='nearest')
        
    print('Encontré', int(len(blobs_log)), 'fuentes.')
    
    if blobs_log[1,2] > 0.5*blobs_log[0,2]: #second source bigger than half first
        print('Múltiples fuentes grandes detectadas en la imagen', str(i))


In [ ]:
#¡NO EJECUTAR!  (Toma mucho tiempo)

for i in range(200):

    img = skimage.io.imread('Image_'+str(i)+'.png')

    image_gray = rgb2gray(img)

    blobs_log = blob_log(image_gray, max_sigma=30, num_sigma=10, threshold=.1)

    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * np.sqrt(2)
    
    blobs_log = blobs_log[blobs_log[:,2].argsort()[::-1]]
    
    X, Y = np.ogrid[:img.shape[0], :img.shape[1]]
    
    center = np.array([img.shape[0]/2, img.shape[1]/2]) #center
    
    for blob in blobs_log:    
        y, x, r = blob    
#        c = plt.Circle((x, y), r, color = 'yellow', linewidth=2, fill=False)
#        ax.add_patch(c)
        
        if (np.linalg.norm(np.array([x,y])-center)) > 10: #If not in center
        
            mask = (X - blob[0])**2 + (Y - blob[1])**2 < r**2
            img[mask] = 0
    
    skimage.io.imsave('NoSources_Image_'+str(i)+'.png',img)
    
    if np.mod(i, 10) == 0:
        print('Procesando la imagen', i)

## ¡Empieza aqui!

In [ ]:
#Toma < 1 minuto

#Leamos las imágenes y redimensionémoslas a algo un poco más manejable.

images = []

for i in range(200):
    img =skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(i)+'.png')
    img_resized = resize(img,(100,100))
    length = np.prod(img_resized.shape)
    img_resized = np.reshape(img_resized,length)
    images.append(img_resized)
    
images = np.vstack(images)

In [ ]:
images.shape #Dimensiones de la matriz. 

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(ax.shape[0]):

    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(i)+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])


In [ ]:
from sklearn.cluster import KMeans # K-Medias (en inglés Kmeans)
kmeans = KMeans(n_clusters=2) # K-Medias (en inglés Kmeans)
kmeans.fit(images)
y_kmeans = kmeans.predict(images)

En este caso, las predicciones (el grupo al que pertenece cada imagen) solo pueden asumir el valor 0 y 1. Aquí mostramos una forma rápida de mostrar qué galaxias se predice que pertenecen a cada grupo.

In [ ]:
print(len(np.where([y_kmeans == 0])[1]))

### Registro de aprendizaje

P: Similar a la línea anterior, ¿cómo puedes imprimir las galaxias que pertenecen al grupo 1? <i>(¡Prueba tu código en la celda de abajo!)</i>

In [ ]:
#  Ingrese el código en esta celda



<details>
<summary style="display: list-item;">¡Haga clic aquí para la respuesta!</summary>
<p>

```python
print(len(np.where([y_kmeans == 1])[1]))
```
    
</p>
</details>

Podemos usar el siguiente código para observar 25 galaxias que se colocaron en el primer grupo y ver si se parecen de alguna manera.



In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans == 0])[1]),25)):
    #Note: the line below selects galaxies that are assigned to cluster 0
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 0])[1][i])+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

Podemos hacer lo mismo para el segundo grupo.

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans == 1])[1]),25)):
    #Note: the line below selects galaxies that are assigned to cluster 1
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 1])[1][i])+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

Ahora hagamos lo mismo pero con tres grupos y condiciones iniciales ligeramente más inteligentes.

In [ ]:
kmeans = KMeans(n_clusters=3, init= 'k-means++')
kmeans.fit(images)
y_kmeans = kmeans.predict(images)

In [ ]:
#Veamos qué tan grandes son los grupos.

for i in range(3):
    print(len(np.where([y_kmeans == i])[1]))

Podemos investigar el grupo pequeño.

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans == 2])[1]),25)): #cambie el índice aquí según sea necesario
    #Nota: la línea de abajo selecciona galaxias que están asignadas al grupo 0
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 2])[1][i])+'.png') #y aquí
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

### Registro de aprendizaje
    
P: ¿Cuál es la similitud entre las imágenes de este grupo?
<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para la respuesta!</summary>
<p>
Parece que este grupo contiene imágenes con artefactos, o donde hay fuentes muy múltiples o grandes descentradas.

Ahora podemos echar un vistazo a los otros dos grupos.

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5, figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans == 0])[1]),25)):
    #Note: the line below selects galaxies that are assigned to cluster 1
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 0])[1][i])+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5, figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans == 1])[1]),25)):
    #Note: the line below selects galaxies that are assigned to cluster 2
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 1])[1][i])+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

### Problema: básicamente, K-Medias (en inglés Kmeans) está agrupando galaxias según su tamaño.

### Learning Check-in
    
P: ¿Por qué K-Medias agrupa las galaxias según su tamaño?

<br>

<details>
<summary style="display: list-item;">¡Haga clic aquí para la respuesta!</summary>
<p>
La distancia euclidiana a los centroides no es invariable en tamaño ni en rotación y, por lo tanto, no es un gran indicador de forma (o color); clasificará como objetos "similares" donde un área similar de la imagen se compone de píxeles brillantes (sobre un fondo oscuro).


Aquí hay algunas posibles ideas para mejorar:

<br> 0. Cualquier métrica debe ser rotacionalmente invariable (o el conjunto de datos debe ser preprocesado para corregir la orientación).

<br> 1. Recortar la imagen central y normalizar el tamaño.

<br> 2. Mejorar el procesamiento previo de múltiples fuentes.

<br> 3. Modificar la métrica de evaluación para otorgar mayor peso a características como el color.

<br> 4. Usar muchos grupos hasta que se obtenga un buen grado de similitud entre los miembros de los mismos grupos, luego volver a agruparlos manualmente.


In [ ]:
#Esto divide uno de los grupos (donde y_means = 2) en dos.

kmeans = KMeans(n_clusters=2, init= 'k-means++')

kmeans.fit(images[y_kmeans == 1])

y_kmeans_l = kmeans.predict(images[y_kmeans == 1])

Veamos los dos nuevos grupos.

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5, figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans_l == 0])[1]),25)):
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 1])[1][np.where([y_kmeans_l == 0])[1]][i])+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

In [ ]:
fig, axes = plt.subplots(ncols= 5, nrows = 5,figsize=(50,50))

ax = axes.ravel()

for i in range(min(len(np.where([y_kmeans_l == 1])[1]),25)):
    #Note: the line below selects galaxies that are assigned to cluster 1
    img = skimage.io.imread('../data/no_sources_images/NoSources_Image_'+str(np.where([y_kmeans == 1])[1][np.where([y_kmeans_l == 1])[1]][i])+'.png')
    ax[i].imshow(img, cmap='gray')
    ax[i].set_xticks([])
    ax[i].set_yticks([])

### Conclusiones

Los algoritmos de agrupamiento pueden dar resultados poco confiables si no se supervisan (juego de palabras intencionado).

El preprocesamiento parece ser bastante importante; definir una métrica de distancia adecuada también puede ayudar.
